In [1]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../../../../')
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from data.engines.model import Record2
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list
from optimization.bayes_optimization_xgb import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

from data.engines.sqlengine import SQLEngine

weekly = 's1'  # s1, s2,  s3, s4, s5, s_1
record_engine = SQLEngine('sqlite:///./{}/real_tune_record_without_alpha-Copy1.db'.format(weekly))


In [2]:
# ref_date_pre = '2019-12-27'

previous_pos = record_engine.fetch_record_meta(Record2, trade_date=None)
previous_pos.trade_date = previous_pos.trade_date.apply(lambda x: str(x))
# previous_pos.trade_date = previous_pos.trade_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

In [3]:
list(set(previous_pos['trade_date']))

['2019-12-10',
 '2019-12-11',
 '2020-01-10',
 '2019-12-13',
 '2019-12-27',
 '2020-01-03',
 '2019-12-16',
 '2019-12-03',
 '2019-12-12',
 '2019-12-17']

In [4]:
a = ['2020-01-10','2020-01-03', '2019-12-27']
ps = pd.DataFrame()
for i in a:
    print(i)
    print(len(previous_pos[previous_pos.trade_date == i]))
    ps = ps.append(previous_pos[previous_pos.trade_date == i])


2020-01-10
100
2020-01-03
100
2019-12-27
100


In [5]:
len(ps)

300

In [6]:
record_engine.write_data('pos_record', ps)

In [7]:
previous_pos = record_engine.fetch_record_meta(Record2, trade_date=None)


In [8]:
len(previous_pos)

4100